<h2>Chicago Data Pull</h2>

In [60]:
import pandas as pd
from sodapy import Socrata
import re
import datetime

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofchicago.org,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("m6dm-c72p", limit=120000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
display(results_df.head())
interested = results_df[['dropoff_centroid_latitude','dropoff_centroid_longitude','dropoff_centroid_location']]
display(interested.head())

interested.to_csv('test_taxi_data.csv', index=False)

,additional_charges,dropoff_census_tract,dropoff_centroid_latitude,dropoff_centroid_location,dropoff_centroid_longitude,dropoff_community_area,fare,pickup_census_tract,pickup_centroid_latitude,pickup_centroid_location,...,pickup_community_area,shared_trip_authorized,tip,trip_end_timestamp,trip_id,trip_miles,trip_seconds,trip_start_timestamp,trip_total,trips_pooled
0,2.5,17031841900,41.8679024175,"{'type': 'Point', 'coordinates': [-87.64295866...",-87.6429586652,28,17.5,17031031200,41.9727095466,"{'type': 'Point', 'coordinates': [-87.65734110...",...,3,False,0,2018-12-12T09:45:00.000,a4eb95b99919771fcd52dc7f712ad5018acf694c,9.35481472996651,1799,2018-12-12T09:15:00.000,20,1
1,2.5,NaN,41.7795828877,"{'type': 'Point', 'coordinates': [-87.76851084...",-87.768510849,64,7.5,NaN,41.7929819032,"{'type': 'Point', 'coordinates': [-87.72420819...",...,62,False,0,2018-12-11T16:45:00.000,a4eb95e4fca1109da5c757174c9f9e50f12930ea,2.30738216270896,828,2018-12-11T16:30:00.000,10,1
2,2.5,17031243500,41.8926581076,"{'type': 'Point', 'coordinates': [-87.65253448...",-87.6525344838,24,5,17031081800,41.8932163595,"{'type': 'Point', 'coordinates': [-87.63784420...",...,8,False,0,2018-11-21T23:45:00.000,a4eb9dfd2fa10a16204461ce4551960a154d13fa,1.15983027182492,379,2018-11-21T23:45:00.000,7.5,1
3,2.5,17031063303,41.9346504482,"{'type': 'Point', 'coordinates': [-87.63464787...",-87.6346478774,6,7.5,17031832500,41.9218774613,"{'type': 'Point', 'coordinates': [-87.66407823...",...,7,False,0,2018-12-01T21:00:00.000,a4eb9ea9d4627763596741e29b1e9fa0327f6c4f,2.40222220914942,646,2018-12-01T21:00:00.000,10,1
4,2.5,17031081201,41.8991556134,"{'type': 'Point', 'coordinates': [-87.62621053...",-87.6262105324,8,5,17031081401,41.8950334495,"{'type': 'Point', 'coordinates': [-87.61971067...",...,8,False,1,2018-11-06T22:30:00.000,a4eb9ff570abb282af9388640108c8cbd97ca7ef,0.8954165521621,348,2018-11-06T22:30:00.000,8.5,1


,dropoff_centroid_latitude,dropoff_centroid_longitude,dropoff_centroid_location
0,41.8679024175,-87.6429586652,"{'type': 'Point', 'coordinates': [-87.64295866..."
1,41.7795828877,-87.768510849,"{'type': 'Point', 'coordinates': [-87.76851084..."
2,41.8926581076,-87.6525344838,"{'type': 'Point', 'coordinates': [-87.65253448..."
3,41.9346504482,-87.6346478774,"{'type': 'Point', 'coordinates': [-87.63464787..."
4,41.8991556134,-87.6262105324,"{'type': 'Point', 'coordinates': [-87.62621053..."


In [61]:
#get start time and pickup

data1 = results_df[['trip_id','trip_start_timestamp','pickup_centroid_latitude','pickup_centroid_longitude']]
data1 = data1.dropna()
print(data1.shape)
data1.head()

(112075, 4)


,trip_id,trip_start_timestamp,pickup_centroid_latitude,pickup_centroid_longitude
0,a4eb95b99919771fcd52dc7f712ad5018acf694c,2018-12-12T09:15:00.000,41.9727095466,-87.6573411073
1,a4eb95e4fca1109da5c757174c9f9e50f12930ea,2018-12-11T16:30:00.000,41.7929819032,-87.7242081939
2,a4eb9dfd2fa10a16204461ce4551960a154d13fa,2018-11-21T23:45:00.000,41.8932163595,-87.6378442095
3,a4eb9ea9d4627763596741e29b1e9fa0327f6c4f,2018-12-01T21:00:00.000,41.9218774613,-87.6640782395
4,a4eb9ff570abb282af9388640108c8cbd97ca7ef,2018-11-06T22:30:00.000,41.8950334495,-87.6197106717


In [62]:
# split trip_start_timestamp column into 'day' and 'time' columns

timestamp_column = data1[['trip_start_timestamp']]
#using [] returns series, using [[]] returns a dataframe 
timestamp_list = timestamp_column.values.tolist()

hourslist = []
dayslist = []
radiallist = []

for i in range(len(timestamp_list)):
    x = re.search('\d{2}:\d{2}:\d{2}', timestamp_list[i][0])
    #x returns <re.Match object; span=(11, 19), match='19:45:00'>
    #x[0] returns 19:45:00 and x[0][:2] returns 19
    radialtime = datetime.datetime.strptime(x[0], '%X').strftime('%I %M %p')
    regulartime = datetime.datetime.strptime(x[0], '%X').strftime('%-I %p')
    #radiallist.append(radialtime[0:5]+radialtime[-3:]) for %X %p local time with AM/PM
    radiallist.append(radialtime[:2]+':'+radialtime[-5:])
    hourslist.append(regulartime)
    
    
    x2 = re.search('\d{4}-\d{2}-\d{2}', timestamp_list[i][0])
    #x2 returns <re.Match object; span=(0, 10), match='2019-05-30'>
    #x2[0] returns date in format 2019-05-30
    dayname = datetime.datetime.strptime(x2[0], '%Y-%m-%d').strftime('%A')
    #convert date from 2019-05-30 format to day of the week
    #strptime is character code input format, strftime character code desired output format
    dayslist.append(dayname)

time_of_day = pd.DataFrame(hourslist, columns=['hours'])
day_of_week = pd.DataFrame(dayslist, columns=['days'])
radial_time_of_day = pd.DataFrame(radiallist, columns=['radial'])

display(radial_time_of_day.head()) 
print('')
display(time_of_day.head()) 
print('')
display(day_of_week.head())

,radial
0,09:15 AM
1,04:30 PM
2,11:45 PM
3,09:00 PM
4,10:30 PM


,hours
0,9 AM
1,4 PM
2,11 PM
3,9 PM
4,10 PM


,days
0,Wednesday
1,Tuesday
2,Wednesday
3,Saturday
4,Tuesday


In [63]:
#to weekday/weekend
display(day_of_week.head())
print('')

day_of_week = day_of_week.replace(['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday'], 'Weekday')
day_of_week = day_of_week.replace(['Saturday','Sunday'], 'Weekend')

display(day_of_week.head())


,days
0,Wednesday
1,Tuesday
2,Wednesday
3,Saturday
4,Tuesday


,days
0,Weekday
1,Weekday
2,Weekday
3,Weekend
4,Weekday


In [64]:
# combining dataframes to make final dataframe and save it to csv for tableau

data1 = data1.drop(['trip_start_timestamp'], axis=1)
data1['time_of_day'] = time_of_day
data1['day_of_week'] = day_of_week
data1 = data1.dropna()
print(data1.shape)
data1.to_csv('pickups and time.csv', index=False)

(104696, 5)


In [65]:
data1['radial_time_of_day'] = radial_time_of_day
data1 = data1.dropna()
print(data1.shape)
display(data1.head())

radialData = data1.groupby(['radial_time_of_day','day_of_week']).count()
radialData = radialData[['trip_id']]
print('')
display(radialData)
radialData.to_csv('radial pickups and time.csv')

(104696, 6)


,trip_id,pickup_centroid_latitude,pickup_centroid_longitude,time_of_day,day_of_week,radial_time_of_day
0,a4eb95b99919771fcd52dc7f712ad5018acf694c,41.9727095466,-87.6573411073,9 AM,Weekday,09:15 AM
1,a4eb95e4fca1109da5c757174c9f9e50f12930ea,41.7929819032,-87.7242081939,4 PM,Weekday,04:30 PM
2,a4eb9dfd2fa10a16204461ce4551960a154d13fa,41.8932163595,-87.6378442095,11 PM,Weekday,11:45 PM
3,a4eb9ea9d4627763596741e29b1e9fa0327f6c4f,41.9218774613,-87.6640782395,9 PM,Weekend,09:00 PM
4,a4eb9ff570abb282af9388640108c8cbd97ca7ef,41.8950334495,-87.6197106717,10 PM,Weekday,10:30 PM


trip_id
radial_time_of_day day_of_week         
01:00 AM           Weekday          263
                   Weekend          517
01:00 PM           Weekday          716
                   Weekend          444
01:15 AM           Weekday          213
                   Weekend          479
01:15 PM           Weekday          734
                   Weekend          446
01:30 AM           Weekday          175
                   Weekend          422
01:30 PM           Weekday          749
                   Weekend          460
01:45 AM           Weekday          178
                   Weekend          381
01:45 PM           Weekday          737
                   Weekend          384
02:00 AM           Weekday          144
                   Weekend          387
02:00 PM           Weekday          683
                   Weekend          420
02:15 AM           Weekday          108
                   Weekend          281
02:15 PM           Weekday          691
                   Weekend          429
02:30 AM           Weekday           97
                   Weekend          252
02:30 PM           Weekday          732
                   Weekend          447
02:45 AM           Weekday           85
                   Weekend          244
...                                 ...
11:00 PM           Weekday          836
                   Weekend          467
11:15 AM           Weekday          655
                   Weekend          380
11:15 PM           Weekday          797
                   Weekend          434
11:30 AM           Weekday          766
                   Weekend          409
11:30 PM           Weekday          742
                   Weekend          473
11:45 AM           Weekday          766
                   Weekend          453
11:45 PM           Weekday          657
                   Weekend          417
12:00 AM           Weekday          383
                   Weekend          595
12:00 PM           Weekday          660
                   Weekend          435
12:15 AM           Weekday          342
                   Weekend          574
12:15 PM           Weekday          717
                   Weekend          442
12:30 AM           Weekday          313
                   Weekend          492
12:30 PM           Weekday          674
                   Weekend          392
12:45 AM           Weekday          241
                   Weekend          482
12:45 PM           Weekday          762
                   Weekend          424

[192 rows x 1 columns]